In [1]:
import pyodbc
import pandas as pd
from boxsdk import JWTAuth, Client
from datetime import datetime, timezone
import os
import logging
from logging.handlers import TimedRotatingFileHandler

conn_str = (
    r'Driver={ODBC Driver 17 for SQL Server};'
    r'Server=dca1-up-dls;'
    r'Database=IACOE;'
    r'Trusted_Connection=yes;'
    )

cnxn = pyodbc.connect(conn_str)

cursor = cnxn.cursor()

query = """SELECT *, ROUND(CAST(e.predicted_true_count AS DECIMAL(18,2)) / e.total_volume,2) AS accuracy
FROM (SELECT
	[Agent CAT Name],
	COUNT(CASE WHEN [Agent CAT Name] IS NOT NULL THEN [Agent CAT Name] END) AS total_volume,
 	COUNT(CASE WHEN [Accuracy] = 1 AND [Agent CAT Name] IS NOT NULL THEN [Agent CAT Name] END) AS predicted_true_count
FROM TaxML_Validation_Data
GROUP BY [Agent CAT Name]) AS e
WHERE e.predicted_true_count >= 1
ORDER BY e.predicted_true_count DESC """

data = pd.read_sql(query,cnxn)
#cnxn.commit()
#cnxn.close()

In [2]:
data.to_csv('Accuracysheet_8Dec2022.tsv', sep="\t")

In [4]:
import pyodbc
import pandas as pd
from boxsdk import JWTAuth, Client
from datetime import datetime, timezone
import os
import logging
from logging.handlers import TimedRotatingFileHandler

conn_str = (
    r'Driver={ODBC Driver 17 for SQL Server};'
    r'Server=dca1-up-dls;'
    r'Database=IACOE;'
    r'Trusted_Connection=yes;'
    )

cnxn = pyodbc.connect(conn_str)

cursor = cnxn.cursor()
query1 = """SELECT *, ROUND(CAST(e.predicted_true_count AS DECIMAL(18,2)) / e.total_volume,2) AS accuracy
FROM (SELECT
    [Establishment_type],[ML CAT Name],
	COUNT(CASE WHEN [Agent CAT Name] IS NOT NULL THEN [Agent CAT Name] END) AS total_volume,
 	COUNT(CASE WHEN [Agent CAT Name] IS NOT NULL AND [Accuracy] = 1 THEN [Agent CAT Name] END) AS predicted_true_count
FROM TaxML_Validation_Data
GROUP BY [Establishment_type],[ML CAT Name]) AS e
WHERE e.predicted_true_count >= 1
ORDER BY e.predicted_true_count DESC """
databyETMLCat = pd.read_sql(query1,cnxn)

In [6]:
databyETMLCat.to_csv('databyETMLCat.tsv', sep="\t")

In [9]:
query2 = """SELECT [Establishment_type],[ML CAT Name],[Agent CAT Name], COUNT(CASE WHEN [Agent CAT Name] IS NOT NULL THEN [Agent CAT Name] END) AS total_volume FROM TaxML_Validation_Data           
            GROUP BY [Establishment_type],[ML CAT Name],[Agent CAT Name] """
query3 = """SELECT [Establishment_type],[ML CAT Name],[Agent CAT Name], COUNT(CASE WHEN [Agent CAT Name] IS NOT NULL AND [Accuracy] = 1 THEN [Agent CAT Name] END) AS predicted_true FROM TaxML_Validation_Data           
            GROUP BY [Establishment_type],[ML CAT Name],[Agent CAT Name] """
databyETMLCatACat1= pd.read_sql(query2,cnxn)
databyETMLCatACat2= pd.read_sql(query3,cnxn)
databyETMLCatACat1.to_csv('databyETMLCatACat1_8Dec2022.tsv', sep="\t")
databyETMLCatACat2.to_csv('databyETMLCatACat2_8Dec2022.tsv', sep="\t")

In [5]:
data.head()

,Agent CAT Name,total_volume,predicted_true_count,accuracy
0,"CAT_PREPARED_FOOD,TEMP_HEATED",247402,246071,0.99
1,CAT_LIQUOR,89583,89566,1.00
2,"CAT_SOFT_DRINK,CONTAINER_BOTTLED,TEMP_COLD",60748,60361,0.99
3,"CAT_PREPARED_DRINK,TEMP_UNHEATED",57593,57425,1.00
4,"CAT_PREPACKAGED_FOOD,TEMP_COLD",54643,54619,1.00


In [6]:
# Create Filename based on timestamp
'''
# Create Filename based on timestamp
FILENAME = 'taxml_accuracysheet'
datetime_str = datetime.now(timezone.utc).strftime("%Y%m%d%H%M%S")
# print(datetime_str)

FILENAME = FILENAME + "_" + datetime_str+".csv"

print(FILENAME)

# create a path
#PATH = os.path.join('data', FILENAME)
PATH = os.path.join(FILENAME)
df.to_csv(PATH, index=False)
'''

FILENAME = 'taxml_accuracysheet'
datetime_str = datetime.now(timezone.utc).strftime("%Y%m%d%H%M%S")
# print(datetime_str)

FILENAME = FILENAME + "_" + datetime_str+".csv"
data.to_csv(FILENAME, index=False)

In [5]:
# Folder ID in box to upload the data
FOLDER_ID = '181092156148'

# establish a connection for BOX
config = JWTAuth.from_settings_file('Tax_Auto_Config.json')
client = Client(config)

In [8]:
FILENAME = FILENAME
# fetch all the items from the folder
folder = client.folder(folder_id=FOLDER_ID)
items = folder.get_items()

flag = False

#logger.debug('Uploading accuracy calcualted file in BOX....')

for item in items:
    # if file name exists, update the content
    if item.name == FILENAME:
        updated_file = client.file(item.id).update_contents(item.name)
        print('File "{0}" has been updated'.format(updated_file.name))
        flag = True
        break
PATH = "taxml_accuracysheet_20221129045955.csv"
# if file doesn not exists upload the file
if not flag:
    #uploaded_file = folder.upload(PATH)
    uploaded_file = folder.upload(PATH, file_name=None, file_description=None,preflight_check=False, preflight_expected_size=0)
    print('File "{0}" has been uploaded'.format(uploaded_file.name))
# replace np.nan by empty string as MYSQL doesn't accept it
#df_final = df_final.replace(np.nan, "")
#logger.debug('Updating MYSQL status....')


File "taxml_accuracysheet_20221129045955.csv" has been uploaded


In [ ]:
def upload_file(client):
    root_folder = client.folder(folder_id='0')
    file_path = os.path.join(os.path.dirname(os.path.realpath(__file__)), 'file.txt')
    a_file = root_folder.upload(file_path, file_name='i-am-a-file.txt')
    try:
        print('{0} uploaded: '.format(a_file.get()['name']))
    finally:
        print('Delete i-am-a-file.txt succeeded: {0}'.format(a_file.delete()))

In [ ]:
To upload a file from a path on disk, call the folder.upload(file_path, file_name=None, file_description=None,preflight_check=False, preflight_expected_size=0)